# Circuit simulation examples (with synaptic manipulations)

In [1]:
import obi_one as obi

In [2]:
# circuit_path_prefix = "/Users/james/Documents/obi/additional_data/O1_data/O1_data/"
# circuit_path_prefix = "/Users/mwr/Documents/artefacts/SONATA/O1_data/"
circuit_path_prefix = "/Users/pokorny/Data/Circuits/"

### Loading two circuits

In [5]:
circuit = obi.Circuit(name="ToyCircuit-S1-6k", path=circuit_path_prefix + "ToyCircuit-S1-6k/circuit_config.json")
# circuit = obi.Circuit(name="O1", path=circuit_path_prefix + "circuit_config_postfix.json")
print(f"Circuit '{circuit}' with {circuit.sonata_circuit.nodes.size} neurons and {circuit.sonata_circuit.edges.size} synapses")

Circuit 'ToyCircuit-S1-6k' with 5924 neurons and 568717 synapses


### Set up simulation campaign

In [ ]:
# Sim duration
sim_duration = 3000.0

# Empty Simulation Configuration
sim_conf = obi.SimulationsForm.empty_config()

# Info
info = obi.Info(campaign_name="O1 Simulation", campaign_description="Simulation of O1 circuit with predefined neuron set and constant current stimulus")
sim_conf.set(info, name="info")

# Neuron Sets
sim_neuron_set = obi.IDNeuronSet(neuron_ids=obi.NamedTuple(name="IDNeuronSet1", elements=range(10)))
sim_conf.add(sim_neuron_set, name='L1All')

sync_neuron_set = obi.IDNeuronSet(neuron_ids=obi.NamedTuple(name="IDNeuronSet2", elements=range(3)))
sim_conf.add(sync_neuron_set, name='ID3')

# Regular Timesteps
regular_timestamps = obi.RegularTimestamps(start_time=0.0, number_of_repetitions=3, interval=sim_duration)
sim_conf.add(regular_timestamps, name='RegularTimestamps')

# Stimulus
poisson_input = obi.PoissonSpikeStimulus(duration=800.0, timestamps=regular_timestamps.ref, frequency=20, source_neuron_set=sim_neuron_set.ref, targeted_neuron_set=sim_neuron_set.ref)
sim_conf.add(poisson_input, name='PoissonInputStimulus')

sync_input = obi.FullySynchronousSpikeStimulus(timestamps=regular_timestamps.ref, source_neuron_set=sync_neuron_set.ref, targeted_neuron_set=sim_neuron_set.ref)
sim_conf.add(sync_input, name='SynchronousInputStimulus')

# Recordings
voltage_recording = obi.SomaVoltageRecording(neuron_set=sim_neuron_set.ref, start_time=0.0, end_time=sim_duration)
sim_conf.add(voltage_recording, name='VoltageRecording')

# Synaptic manipulations (executed in order!!)
syn_manip_mg = obi.SynapticMgManipulation(magnesium_value=[2.0, 2.4])
syn_manip_use = obi.ScaleAcetylcholineUSESynapticManipulation(use_scaling=0.7050728631217412)
sim_conf.add(syn_manip_mg, name='SynapticMgManipulation')
sim_conf.add(syn_manip_use, name='ScaleAcetylcholineUSESynapticManipulation')

# Initialization
simulations_initialize = obi.SimulationsForm.Initialize(circuit=circuit, 
                                                        node_set=sim_neuron_set.ref, 
                                                        simulation_length=sim_duration)
sim_conf.set(simulations_initialize, name='initialize')

# Validated Config
validated_sim_conf = sim_conf.validated_config()

In [9]:
# sim_conf.model_dump(mode="json")

In [10]:
grid_scan = obi.GridScan(form=validated_sim_conf, output_root='../../obi-output/circuit_simulations_with_manipulations/grid_scan')
grid_scan.multiple_value_parameters(display=True)
grid_scan.coordinate_parameters(display=True)
grid_scan.execute(processing_method='generate')

[2025-06-27 12:00:36,746] INFO: 
MULTIPLE VALUE PARAMETERS
[2025-06-27 12:00:36,747] INFO: manipulations.SynapticMgManipulation.magnesium_value: [2.0, 2.4]
[2025-06-27 12:00:36,747] INFO: 
COORDINATE PARAMETERS
[2025-06-27 12:00:36,748] INFO: manipulations.SynapticMgManipulation.magnesium_value: 2.0
[2025-06-27 12:00:36,748] INFO: manipulations.SynapticMgManipulation.magnesium_value: 2.4
[2025-06-27 12:00:36,748] INFO: None
[2025-06-27 12:00:36,750] INFO: create_bbp_workflow_campaign_config() not yet complete.
[2025-06-27 12:00:36,753] INFO: initialize.circuit is a Circuit instance.
[2025-06-27 12:00:36,770] INFO: initialize.circuit is a Circuit instance.


In [11]:
# Deserialization
grid_scan_ds = obi.deserialize_obi_object_from_json_file("../../obi-output/circuit_simulations_with_manipulations/grid_scan/run_scan_config.json")

In [12]:
grid_scan_ds

GridScan(type='GridScan', form=SimulationsForm(type='SimulationsForm', timestamps={'RegularTimestamps': RegularTimestamps(type='RegularTimestamps', start_time=0.0, number_of_repetitions=3, interval=3000.0)}, stimuli={'PoissonInputStimulus': PoissonSpikeStimulus(type='PoissonSpikeStimulus', timestamp_offset=0.0, timestamps=TimestampsReference(type='TimestampsReference', block_dict_name='timestamps', block_name='RegularTimestamps'), source_neuron_set=NeuronSetReference(type='NeuronSetReference', block_dict_name='neuron_sets', block_name='L1All'), targeted_neuron_set=NeuronSetReference(type='NeuronSetReference', block_dict_name='neuron_sets', block_name='L1All'), duration=800.0, random_seed=0, frequency=20.0), 'SynchronousInputStimulus': FullySynchronousSpikeStimulus(type='FullySynchronousSpikeStimulus', timestamp_offset=0.0, timestamps=TimestampsReference(type='TimestampsReference', block_dict_name='timestamps', block_name='RegularTimestamps'), source_neuron_set=NeuronSetReference(type='